In [1]:
!pip install tensorflow-io==0.25.0
!pip install kafka-python
!pip install tensorflow==2.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.34.0
    Uninstalling tensorflow-io-gcs-filesystem-0.34.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.34.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio

In [3]:
print("tensorflow-io version: {}".format(tfio.__version__))
print("tensorflow version: {}".format(tf.__version__))

tensorflow-io version: 0.25.0
tensorflow version: 2.8.0


In [4]:
!curl -sSOL https://downloads.apache.org/kafka/3.6.1/kafka_2.13-3.6.1.tgz
!tar -xzf kafka_2.13-3.6.1.tgz

In [5]:
!./kafka_2.13-3.6.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.6.1/config/zookeeper.properties
!./kafka_2.13-3.6.1/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.6.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10


Waiting for 10 secs until kafka and zookeeper services are up and running


In [6]:
!ps -ef | grep kafka

root        2184     357  0 15:11 ?        00:00:00 /bin/bash -c ps -ef | grep kafka
root        2186    2184  0 15:11 ?        00:00:00 grep kafka


In [7]:
!./kafka_2.13-3.6.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic bigmartsale4-train
!./kafka_2.13-3.6.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 --topic bigmartsale4-test


Created topic bigmartsale4-train.
Created topic bigmartsale4-test.


In [9]:
!./kafka_2.13-3.6.1/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic bigmartsale4-train
!./kafka_2.13-3.6.1/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic bigmartsale4-test


Topic: bigmartsale4-train	TopicId: UCAVzrngR6iKzfYcZmlM7w	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: bigmartsale4-train	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
Topic: bigmartsale4-test	TopicId: yVIlUQriQmO0vMgIjLgsww	PartitionCount: 2	ReplicationFactor: 1	Configs: 
	Topic: bigmartsale4-test	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
	Topic: bigmartsale4-test	Partition: 1	Leader: 0	Replicas: 0	Isr: 0


In [10]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=685d195a666da9bd4ad63622526d23bdea7c4185d665f30e692637bab88cd729
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [11]:
!pip install findspark

In [12]:
import pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession

scala_version = '2.12'  # your scala version
spark_version = '3.0.1' # your spark version
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:2.8.0' #your kafka version
]
spark = SparkSession.builder.master("local").appName("kafka-example").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

In [13]:
train=spark.read.csv(r"/content/drive/MyDrive/BT_KAFKA/Train.csv", header=True, inferSchema=True)


In [14]:
from pyspark.sql.functions import col,isnan, when, count
train.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in train.columns]
   ).show()

+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|              0|       1463|               0|              0|        0|       0|                0|                        0|       2410|                   0|          0|                0|
+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+



In [15]:
train=train.na.fill(value=0,subset=["Item_Weight"])

In [16]:
features_encode=['Item_Identifier','Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Location_Type','Outlet_Type']

In [17]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(train) for column in features_encode]
pipeline = Pipeline(stages=indexers)
data_train = pipeline.fit(train).transform(train)

In [18]:
features=['Item_Identifier_index','Item_Weight','Item_Fat_Content_index','Item_Visibility','Item_Type_index','Item_MRP','Outlet_Identifier_index','Outlet_Establishment_Year_index','Outlet_Location_Type_index','Outlet_Type_index']

In [19]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
vectorAssembler = VectorAssembler(inputCols=features, outputCol="unscaled_features")
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
estimators = [vectorAssembler, standardScaler]
pipeline = Pipeline(stages=estimators)
data = pipeline.fit(data_train).transform(data_train)

In [20]:
modeldata=data.select('features','Item_Outlet_Sales')

In [21]:
(train_t,test_t)=modeldata.randomSplit([0.8,0.2])

In [22]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol='features', labelCol='Item_Outlet_Sales',maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrr=lr.fit(train_t)

In [23]:
y_pred = lrr.transform(test_t)
y_pred.show()

+--------------------+-----------------+------------------+
|            features|Item_Outlet_Sales|        prediction|
+--------------------+-----------------+------------------+
|(10,[0,2,3,5,9],[...|        5375.0034| 4859.543644690989|
|(10,[0,2,3,5,9],[...|        5087.3778|  4129.28220242423|
|(10,[0,2,3,5,9],[...|        4432.2306| 4506.594491351274|
|(10,[0,2,3,5,9],[...|        5294.4416| 4017.363108269464|
|(10,[0,2,3,5,9],[...|        3374.2744|3129.1718217201073|
|(10,[0,2,3,5,9],[...|         966.7416|2570.6121659545693|
|(10,[0,2,3,5,9],[...|        5978.2182| 3287.566761792685|
|(10,[0,2,3,5,9],[...|           2996.1|3499.9840683402062|
|(10,[0,2,3,5,9],[...|        7116.0704| 4707.403944758632|
|(10,[0,2,3,5,9],[...|        4828.3816| 3200.927897036621|
|(10,[0,2,3,5,9],[...|        1543.3244|1837.8347034671335|
|(10,[0,2,3,5,9],[...|           4327.7| 3716.802351384158|
|(10,[0,2,3,5,9],[...|        2143.2102|1975.6315582401235|
|(10,[0,2,3,5,9],[...|         12117.56|

In [24]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol='features', labelCol='Item_Outlet_Sales',maxIter=10)
gbt=gbt.fit(train_t)
y_pred = gbt.transform(test_t)
y_pred.show()

+--------------------+-----------------+------------------+
|            features|Item_Outlet_Sales|        prediction|
+--------------------+-----------------+------------------+
|(10,[0,2,3,5,9],[...|        5375.0034| 6692.580995625761|
|(10,[0,2,3,5,9],[...|        5087.3778| 6389.477028661729|
|(10,[0,2,3,5,9],[...|        4432.2306| 6074.595954785714|
|(10,[0,2,3,5,9],[...|        5294.4416| 6389.477028661729|
|(10,[0,2,3,5,9],[...|        3374.2744|3250.0161022167576|
|(10,[0,2,3,5,9],[...|         966.7416| 2647.765838759359|
|(10,[0,2,3,5,9],[...|        5978.2182|4004.6693794682606|
|(10,[0,2,3,5,9],[...|           2996.1| 3895.376526785505|
|(10,[0,2,3,5,9],[...|        7116.0704| 5593.447293853962|
|(10,[0,2,3,5,9],[...|        4828.3816| 3182.400876460216|
|(10,[0,2,3,5,9],[...|        1543.3244| 843.0843221075398|
|(10,[0,2,3,5,9],[...|           4327.7| 4660.212374869375|
|(10,[0,2,3,5,9],[...|        2143.2102|1887.2261328604354|
|(10,[0,2,3,5,9],[...|         12117.56|

In [25]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol='features', labelCol='Item_Outlet_Sales')
dt=dt.fit(train_t)
y_pred = dt.transform(test_t)
y_pred.show()

+--------------------+-----------------+------------------+
|            features|Item_Outlet_Sales|        prediction|
+--------------------+-----------------+------------------+
|(10,[0,2,3,5,9],[...|        5375.0034|6427.9237309090895|
|(10,[0,2,3,5,9],[...|        5087.3778|6427.9237309090895|
|(10,[0,2,3,5,9],[...|        4432.2306|6427.9237309090895|
|(10,[0,2,3,5,9],[...|        5294.4416|6427.9237309090895|
|(10,[0,2,3,5,9],[...|        3374.2744|3197.5421867924515|
|(10,[0,2,3,5,9],[...|         966.7416| 2652.029966942149|
|(10,[0,2,3,5,9],[...|        5978.2182|4021.0406519083967|
|(10,[0,2,3,5,9],[...|           2996.1|4021.0406519083967|
|(10,[0,2,3,5,9],[...|        7116.0704| 5497.020914838708|
|(10,[0,2,3,5,9],[...|        4828.3816|3197.5421867924515|
|(10,[0,2,3,5,9],[...|        1543.3244|1025.0425217391305|
|(10,[0,2,3,5,9],[...|           4327.7| 4696.366776000001|
|(10,[0,2,3,5,9],[...|        2143.2102|1890.6442263157894|
|(10,[0,2,3,5,9],[...|         12117.56|

In [26]:
import numpy as np
from json import dumps
kafka_server="127.0.0.1:9092"
producer = KafkaProducer(bootstrap_servers=kafka_server,value_serializer = lambda x:dumps(x).encode('utf-8'))


In [27]:
test=pd.read_csv('/content/drive/MyDrive/BT_KAFKA/Test.csv')
test=test.drop('Outlet_Size',axis=1)

In [28]:
test=np.array(test)
topic_name="bigmartsale4-test"
for i in test:
  a=np.array2string(i, precision=8, separator=',',
                      suppress_small=True)
  a=a.replace('\n','')
  producer.send(topic_name, value=a)

In [30]:
kafkaDf = spark.read.format("kafka").option("kafka.bootstrap.servers", kafka_server).option("subscribe", topic_name).option("startingOffsets", "earliest").option("max.poll.interval.ms", 300000).load()
from time import sleep
from IPython.display import display, clear_output
import pyspark.sql.functions as f
batchDF = kafkaDf.selectExpr("cast(value as string) as value")
from pyspark.sql.functions import split, col,substring,regexp_replace
batchDF=batchDF.select("value", f.translate(f.col("value"), '"[', "").alias("values"))
batchDF=batchDF.select("values", f.translate(f.col("values"), 'nan', "0").alias("valuess"))
test = batchDF.withColumn('Item_Identifier', split(batchDF['valuess'], ',').getItem(0))\
                .withColumn("Item_Weight", split(batchDF['valuess'], ',').getItem(1).cast('float'))\
                .withColumn("Item_Fat_Content", split(batchDF['valuess'], ',').getItem(2))\
                .withColumn("Item_Visibility", split(batchDF['valuess'], ',').getItem(3).cast('float'))\
                .withColumn("Item_Type", split(batchDF['valuess'], ',').getItem(4))\
                .withColumn("Item_MRP", split(batchDF['valuess'], ',').getItem(5).cast('float'))\
                .withColumn("Outlet_Identifier", split(batchDF['valuess'], ',').getItem(6))\
                .withColumn("Outlet_Establishment_Year", split(batchDF['valuess'], ',').getItem(7))\
                .withColumn("Outlet_Location_Type", split(batchDF['valuess'], ',').getItem(8))\
                .withColumn("Outlet_Type", split(batchDF['valuess'], ',').getItem(9))
test=test.drop('values','valuess')

In [31]:
test.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+--------------------+--------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Location_Type|         Outlet_Type|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+--------------------+--------------------+
|        'FDY38'|        0.0|        'Regulr'|     0.11859931|              'Diry'|  234.23|         'OUT027'|                     1985|            'Tier 3'| 'Supermrket Type3']|
|        'FDH56'|        9.8|        'Regulr'|      0.0638172|'Fruits 0d Vegetb...|117.1492|         'OUT046'|                     1997|            'Tier 1'| 'Supermrket Type1']|
|        'FDC48'|        0.0|        'Low Ft'|    0.015782494|       'Bki0g Goods'| 81.0592|         'OUT

In [32]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(test) for column in features_encode]
pipeline = Pipeline(stages=indexers)
data_test = pipeline.fit(test).transform(test)

In [33]:
data_test.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+--------------------+--------------------+---------------------+----------------------+---------------+-----------------------+-------------------------------+--------------------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Location_Type|         Outlet_Type|Item_Identifier_index|Item_Fat_Content_index|Item_Type_index|Outlet_Identifier_index|Outlet_Establishment_Year_index|Outlet_Location_Type_index|Outlet_Type_index|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+--------------------+--------------------+---------------------+----------------------+---------------+-----------------------+-------------------------------+-----------------------

In [34]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
vectorAssembler = VectorAssembler(inputCols=features, outputCol="unscaled_features")
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
estimators = [vectorAssembler, standardScaler]
pipeline = Pipeline(stages=estimators)
data = pipeline.fit(data_test).transform(data_test)

In [35]:
data.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+--------------------+--------------------+---------------------+----------------------+---------------+-----------------------+-------------------------------+--------------------------+-----------------+--------------------+--------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Location_Type|         Outlet_Type|Item_Identifier_index|Item_Fat_Content_index|Item_Type_index|Outlet_Identifier_index|Outlet_Establishment_Year_index|Outlet_Location_Type_index|Outlet_Type_index|   unscaled_features|            features|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+--------------------+--------------------+---------------------+----------------------+-----------

In [36]:
pre=data.select('features')

In [37]:
y_pred = gbt.transform(pre)
y_pred.show(10)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[1.74957807780796...| 5890.725905482789|
|[2.28164154804546...|1968.3362803936118|
|(10,[0,3,4,5,9],[...|1950.4277050613257|
|[1.44519942591534...|1508.0891281155746|
|[3.18998413912658...| 176.6071515943125|
|[1.80949513526714...|5347.1812611350115|
|[0.02396682298367...| 636.1047783023351|
|[1.41883592063331...| 2629.212531829064|
|[3.54948648388164...|3090.0549764366624|
|[2.10189037566792...|3125.4402277810636|
+--------------------+------------------+
only showing top 10 rows



In [47]:
gbt.write().overwrite().save('/content/drive/MyDrive/BT_KAFKA/GBTRegressor')
lrr.write().overwrite().save( '/content/drive/MyDrive/BT_KAFKA/LinearRegression')
dt.write().overwrite().save( '/content/drive/MyDrive/BT_KAFKA/DecisionTreeRegressor')